### IMPORTING LIBRARIES

In [67]:
import sys
import hashlib
import base64
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad, unpad

In [68]:
ZERO_WIDTH_SPACE = "\u200B"  
ZERO_WIDTH_NON_JOINER = "\u200C"
ZERO_WIDTH_JOINER = "\u200D" 


In [69]:
AUTHORIZED_USERS = {"user" + str(i) for i in range(1, 21)}
ALL_USERS = {"user" + str(i) for i in range(1, 101)}

In [70]:

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()


### AES Encryption

In [71]:
def encrypt_message(secret_message, password):
    key = hashlib.sha256(password.encode()).digest()
    cipher = AES.new(key, AES.MODE_CBC)
    encrypted_bytes = cipher.encrypt(pad(secret_message.encode(), AES.block_size))
    return base64.b64encode(cipher.iv + encrypted_bytes).decode()


### AES Decryption

In [72]:
def decrypt_message(encrypted_message, password):
    try:
        key = hashlib.sha256(password.encode()).digest()
        encrypted_data = base64.b64decode(encrypted_message)
        iv = encrypted_data[:16]
        cipher = AES.new(key, AES.MODE_CBC, iv)
        decrypted_bytes = unpad(cipher.decrypt(encrypted_data[16:]), AES.block_size)
        return decrypted_bytes.decode()
    except Exception:
        return None

###  Encode message using zero-width characters

In [90]:
def encode_message(carrier_text, secret_message, password):
    hashed_password = hash_password(password)
    encrypted_message = encrypt_message(secret_message, password)
    binary_secret = ''.join(format(ord(char), '08b') for char in encrypted_message)
    encoded = ''.join(ZERO_WIDTH_SPACE if bit == '0' else ZERO_WIDTH_NON_JOINER for bit in binary_secret)
    return carrier_text + ZERO_WIDTH_JOINER + hashed_password + encoded

### Decode message for authorized users

In [91]:
def decode_message(stego_text):
    user_id = input("Enter your User ID: ")
    if user_id not in ALL_USERS:
        return "Invalid User ID."
    if user_id not in AUTHORIZED_USERS:
        return stego_text.split(ZERO_WIDTH_JOINER)[0]
    
    hashed_password_start = stego_text.find(ZERO_WIDTH_JOINER)
    if hashed_password_start == -1:
        return stego_text
    
    hashed_password_section = stego_text[hashed_password_start+1:]
    user_password = input("Enter password to access hidden message: ")
    hashed_password = hash_password(user_password)
    
    if hashed_password not in hashed_password_section:
        return stego_text.split(ZERO_WIDTH_JOINER)[0]
    
    encoded_part = hashed_password_section.split(hashed_password, 1)[1]
    binary_secret = ''.join('0' if char == ZERO_WIDTH_SPACE else '1' for char in encoded_part)
    encrypted_message = ''.join(chr(int(binary_secret[i:i+8], 2)) for i in range(0, len(binary_secret), 8))
    
    decrypted_message = decrypt_message(encrypted_message, user_password)
    if decrypted_message:
        return f"Correct Password! Hidden Message: {decrypted_message}"
    else:
        return "Incorrect password or corrupted data."

In [92]:
carrier_text = ("In the heart of a bustling city, where skyscrapers kissed the clouds and the streets buzzed with life, "
           "a story was unfolding. The sun cast long shadows as people hurried to their destinations, unaware of the "
           "hidden messages that surrounded them. Every building, every street sign, and every whisper of the wind carried "
           "secrets waiting to be uncovered. It was a world of coded words, of secrets hidden in plain sight, where only the "
           "keenest minds could decipher the truth. Amidst this, a special message lay embedded within, waiting for the "
           "right eyes to see it.")
secret_message = "Confidential Data: 12345XYZ"
password = "SecureKey"

In [93]:
stego_text = encode_message(carrier_text, secret_message, password)
print("Stego Text (Visible Part):", carrier_text)

Stego Text (Visible Part): In the heart of a bustling city, where skyscrapers kissed the clouds and the streets buzzed with life, a story was unfolding. The sun cast long shadows as people hurried to their destinations, unaware of the hidden messages that surrounded them. Every building, every street sign, and every whisper of the wind carried secrets waiting to be uncovered. It was a world of coded words, of secrets hidden in plain sight, where only the keenest minds could decipher the truth. Amidst this, a special message lay embedded within, waiting for the right eyes to see it.


In [96]:
retrieved_message = decode_message(stego_text)
print(retrieved_message)


Enter your User ID:  user2
Enter password to access hidden message:  SecureKey


Correct Password! Hidden Message: Confidential Data: 12345XYZ
